In [1]:
from QuiltLoader import QuiltLoader as ql

# QuiltLoader can handle both 'org/pkg'
data = ql('aics/random_sample')

# or preload
from quilt.data.aics import random_sample
data = ql(random_sample)

# we have subscriptable nodes now
fov = data['fovs'][19]
img = fov['image']
print(img)

TiffFile '3b2b023d4b2e...0dc874977ed'  362.06 MiB  LittleEndian  OME  455 Pages


In [2]:
# look at those no loads
meta = fov['info']
print(meta.keys())

dict_keys(['plate_type', 'plate_id', 'plate_barcode', 'population_id', 'passage', 'clone', 'parent_plate_id', 'image_ids', 'seeding_density', 'cellbatch', 'source_plate_id', 'cell_viability', 'well_type', 'well_name', 'concentration', 'coating_type', 'seed_date', 'colony_ids', 'location', 'content_type', 'instrument', 'channels', 'plates', 'lines', 'wells', 'cell_segs', 'nuclei_segs', 'structure_segs'])


In [3]:
# iterable nodes too
for i, line in enumerate(data['lines']):
    print(i, line['line'])

0 AICS-10
1 AICS-11
2 AICS-12
3 AICS-13
4 AICS-16
5 AICS-17
6 AICS-23
7 AICS-24
8 AICS-25


In [ ]:
# can even slice
for i, fov in enumerate(data['fovs'][0:6:2]):
    print(i, fov)

0 <GroupNode>
image/
info/

1 <GroupNode>
image/
info/

2 <GroupNode>
image/
info/



In [ ]:
img = ql.display_channels(img)

In [ ]:
ql.display_rgb(img)

In [ ]:
ql.display_rgb(img, use='mean')

In [ ]:
ql.display_rgb(img, use='percentile')

In [ ]:
ql.display_stack(img)

In [ ]:
ql.display_stack(img, use='mean')

In [ ]:
ql.display_stack(img, use='percentile')